# 03_quantum_kernel.ipynb — QSVM (precomputed kernel)

This notebook computes a quantum kernel (via PennyLane) on PCA‑reduced k‑mer features and trains an SVM with the precomputed Gram matrix. Execute sequentially: Cell 0 (perf env) -> Cell 1 ... Cell 7.

# Cell 0 — perf env

In [ ]:
# Normalize thread usage for reproducible classical linear algebra performance
import os
os.environ.setdefault("OMP_NUM_THREADS", "8")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "8")
os.environ.setdefault("MKL_NUM_THREADS", "8")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "8")
print("BLAS threads:", os.environ.get("OMP_NUM_THREADS"), os.environ.get("OPENBLAS_NUM_THREADS"))

# Cell 1 — imports, paths, journaling

In [ ]:
# Imports: classical preprocessing + PennyLane for quantum kernel construction
from pathlib import Path
import json, warnings, time, os
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

import pennylane as qml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix,
    classification_report, balanced_accuracy_score, matthews_corrcoef, average_precision_score
)
from sklearn.svm import SVC

warnings.filterwarnings("ignore")

ROOT = Path(".")
PROCESSED = ROOT / "data" / "processed"
RESULTS = ROOT / "results"
(RESULTS / "kernels").mkdir(parents=True, exist_ok=True)
(RESULTS / "metrics").mkdir(parents=True, exist_ok=True)
(RESULTS / "plots").mkdir(parents=True, exist_ok=True)
(RESULTS / "logs").mkdir(parents=True, exist_ok=True)

np.random.seed(7)

# ---- Run journal (for documentation) ----
class RunJournal:
    def __init__(self): self.events = []
    def log(self, step, status, message, **extras):
        self.events.append({
            "ts": time.strftime("%Y-%m-%d %H:%M:%S"),
            "step": step, "status": status, "message": message, **extras
        })
        sym = "✅" if status=="ok" else ("⚠️" if status=="warn" else "❌")
        print(f"{sym} [{step}] {message}")
    def df(self): return pd.DataFrame(self.events)
    def save(self, base: Path):
        df = self.df()
        md = ["| ts | step | status | message |", "|---|---|---|---|"]
        for _,r in df.iterrows():
            md.append(f"| {r.ts} | {r.step} | {r.status} | {r.message} |")
        (base.with_suffix(".md")).write_text("\n".join(md), encoding="utf-8")
        (base.with_suffix(".json")).write_text(df.to_json(orient="records", indent=2), encoding="utf-8")
        print(f"📝 Saved journal:\n  - {base.with_suffix('.md')}\n  - {base.with_suffix('.json')}")

J = RunJournal()

# Cell 2 — load data & PCA→angles (multi-dataset aware + logging)

In [ ]:
# Try multi-dataset artifacts first; fall back to single-dataset artifacts
enc_candidates = [PROCESSED/"encodings_all.npz", PROCESSED/"encodings.npz"]
spl_candidates = [PROCESSED/"splits_pooled.json", PROCESSED/"splits.json"]

enc_path = next((p for p in enc_candidates if p.exists()), None)
spl_path = next((p for p in spl_candidates if p.exists()), None)

if enc_path is None or spl_path is None:
    if enc_path is None: J.log("load", "fail", "Encodings file not found (tried encodings_all.npz, encodings.npz)")
    if spl_path is None: J.log("load", "fail", "Splits file not found (tried splits_pooled.json, splits.json)")
    raise FileNotFoundError("Required data artifacts missing in data/processed")

data = np.load(enc_path, allow_pickle=True)
with open(spl_path) as f:
    SPL = json.load(f)
J.log("load", "ok", f"Loaded encodings from {enc_path.name} and splits from {spl_path.name}")

y = data["y"].astype(np.int64)
X_kmer = data["kmer"].astype(np.float32)

# Optional per-dataset index
ds_idx = data["ds_idx"] if "ds_idx" in data.files else None
ds_map = None
ds_map_path = PROCESSED/"dataset_index.csv"
if ds_idx is not None and ds_map_path.exists():
    ds_map = pd.read_csv(ds_map_path).set_index("ds_idx")["accession"].to_dict()
    J.log("datasets", "ok", f"Detected {len(set(ds_idx))} dataset(s) with mapping.")
elif ds_idx is not None:
    J.log("datasets", "warn", "ds_idx present but dataset_index.csv missing — names unavailable.")

tr_idx = np.array(SPL["train"]); va_idx = np.array(SPL["val"]); te_idx = np.array(SPL["test"])
pos_rate = float(y.mean()) if len(y) else float("nan")
J.log("splits", "ok", f"train={len(tr_idx)}, val={len(va_idx)}, test={len(te_idx)}, pos_rate={pos_rate:.4f}")

# PCA → standardize → angle map
D = int(os.environ.get("QK_D", "8"))  # number of principal components (== number of qubits)
pca = PCA(n_components=D, random_state=7)
X_tr_p = pca.fit_transform(X_kmer[tr_idx])
X_va_p = pca.transform(X_kmer[va_idx])
X_te_p = pca.transform(X_kmer[te_idx])

ev = pca.explained_variance_ratio_.sum()
J.log("pca", "ok", f"PCA to D={D} components (variance explained={ev:.3f})")

scaler = StandardScaler(with_mean=True, with_std=True)
X_tr_z = scaler.fit_transform(X_tr_p)
X_va_z = scaler.transform(X_va_p)
X_te_z = scaler.transform(X_te_p)

# Angle map with clipping for robustness (document clipping rate)
def to_angles(X, clip=3.0):
    Xc = np.clip(X, -clip, clip)
    return (np.pi * Xc / clip).astype(np.float32), float((np.abs(X) > clip).sum())

Xtr, nclip_tr = to_angles(X_tr_z); Xva, nclip_va = to_angles(X_va_z); Xte, nclip_te = to_angles(X_te_z)
J.log("angles", "ok", f"Angle embedding created; clipped values — train:{int(nclip_tr)}, val:{int(nclip_va)}, test:{int(nclip_te)}")

# Cell 3 — device + kernel circuit (with device reporting)

In [ ]:
# Define embedding circuit and kernel evaluation (overlap) using an adjoint construction
def make_device(n_wires, shots=None):
    try:
        dev_ = qml.device("lightning.qubit", wires=n_wires, shots=shots)
        J.log("device", "ok", f"Using lightning.qubit (wires={n_wires}, shots={shots})")
        return dev_
    except Exception as e:
        J.log("device", "warn", f"lightning.qubit unavailable ({e}); falling back to default.qubit")
        return qml.device("default.qubit", wires=n_wires, shots=shots)

n_wires = int(D)
wires = list(range(n_wires))
dev = make_device(n_wires, shots=None)

def cz_ring(ws):
    n = len(ws)
    for i in range(n):
        qml.CZ(wires=[ws[i], ws[(i+1) % n]])

def U(x):
    qml.AngleEmbedding(x, wires=wires, rotation="Y")
    cz_ring(wires)

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    U(x1)
    qml.adjoint(U)(x2)
    # Fidelity with |0...0> equals |<phi(x1)|phi(x2)>|^2
    return qml.expval(qml.Projector([0]*n_wires, wires=wires))

# Cell 4 — Gram matrices (symmetric speed-up + progress + saves)

In [ ]:
# Build Gram matrices (train-train, val-train, test-train) via pairwise quantum kernel evaluations

def gram_matrix_square(X):
    """Symmetric Gram for XA vs XA (upper-triangular fill)."""
    n = len(X)
    K = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        K[i, i] = kernel_circuit(X[i], X[i])
        for j in range(i+1, n):
            kij = kernel_circuit(X[i], X[j])
            K[i, j] = kij
            K[j, i] = kij
        if (i+1) % 50 == 0 or i == n-1:
            print(f" row {i+1}/{n} ready")
    return K

def gram_matrix_rect(XA, XB):
    """Rectangular Gram for XA vs XB."""
    K = np.zeros((len(XA), len(XB)), dtype=np.float64)
    for i in range(len(XA)):
        for j in range(len(XB)):
            K[i, j] = kernel_circuit(XA[i], XB[j])
        if (i+1) % 50 == 0 or i == len(XA)-1:
            print(f" row {i+1}/{len(XA)} ready")
    return K

# Control kernel cost via truncation
MAX_TRAIN = int(os.environ.get("QK_MAX_TRAIN", "600"))  # set small (e.g., 600) if too slow
sel_tr = tr_idx[: (MAX_TRAIN or len(tr_idx))]
Xtr_sel, ytr_sel = Xtr[:len(sel_tr)], y[sel_tr]
if len(sel_tr) < len(tr_idx):
    J.log("kernel", "warn", f"Training truncated to first {len(sel_tr)} samples for kernel cost control.")

t0 = time.time()
K_trtr = gram_matrix_square(Xtr_sel)
K_vatr = gram_matrix_rect(Xva, Xtr_sel)
K_tetr = gram_matrix_rect(Xte, Xtr_sel)
dt = time.time() - t0
J.log("kernel", "ok", f"Computed Gram matrices in {dt/60:.1f} min (train={K_trtr.shape}, val={K_vatr.shape}, test={K_tetr.shape})")

np.save(RESULTS/"kernels/K_trtr.npy", K_trtr)
np.save(RESULTS/"kernels/K_vatr.npy", K_vatr)
np.save(RESULTS/"kernels/K_tetr.npy", K_tetr)
print("saved gram matrices")

# Cell 5 — QSVM on precomputed kernel (full metrics, plots, docs)

In [ ]:
# Train SVM with precomputed quantum kernel; full metrics & plots

def choose_threshold(y_val, p_val, name="QSVM"):
    grid = np.linspace(0.05, 0.95, 37)
    best_thr, best_f1 = 0.5, -1
    from sklearn.metrics import f1_score
    for t in grid:
        f1 = f1_score(y_val, (p_val >= t).astype(int), zero_division=0)
        if f1 > best_f1:
            best_f1, best_thr = float(f1), float(t)
    if np.isnan(best_f1):
        J.log("threshold", "warn", f"{name}: F1 undefined on val; using default thr=0.5")
        return 0.5
    J.log("threshold", "ok", f"{name}: selected thr={best_thr:.2f} (val F1={best_f1:.3f})")
    return best_thr

def extended_metrics(y_true, y_prob, thr):
    y_hat = (y_prob >= thr).astype(int)
    acc = accuracy_score(y_true, y_hat)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_hat, average="binary", zero_division=0)
    try: auc = roc_auc_score(y_true, y_prob)
    except Exception: auc = float("nan")
    try: ap  = average_precision_score(y_true, y_prob)
    except Exception: ap  = float("nan")
    cm = confusion_matrix(y_true, y_hat, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    tnr = tn / (tn + fp) if (tn + fp) else float("nan")  # specificity
    bal = balanced_accuracy_score(y_true, y_hat)
    mcc = matthews_corrcoef(y_true, y_hat) if len(np.unique(y_true))==2 else float("nan")
    rep = classification_report(y_true, y_hat, output_dict=True, zero_division=0)
    return {
        "acc": acc, "prec": prec, "rec": rec, "f1": f1,
        "roc_auc": auc, "pr_auc": ap, "specificity": tnr,
        "balanced_acc": bal, "mcc": mcc, "thr": thr,
        "tp": int(tp), "tn": int(tn), "fp": int(fp), "fn": int(fn),
        "support": int(len(y_true)),
    }, cm, rep

def save_cm_csv(cm, out_csv, normalized=False):
    if normalized:
        cm = cm.astype(np.float64)
        rs = cm.sum(axis=1, keepdims=True)
        cm = np.divide(cm, np.where(rs==0, 1, rs))
    df = pd.DataFrame(cm, index=["true_0","true_1"], columns=["pred_0","pred_1"])
    df.to_csv(out_csv, index=True)

def plot_roc(y_true, y_prob, title, out_png):
    from sklearn.metrics import roc_curve, auc
    try:
        fpr, tpr, _ = roc_curve(y_true, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(); plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
        plt.plot([0,1],[0,1], linestyle="--")
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(title); plt.legend(loc="lower right"); plt.tight_layout()
        plt.savefig(out_png, dpi=150); plt.close()
    except Exception as e:
        J.log("plot", "warn", f"ROC plot skipped: {e}")

def plot_pr(y_true, y_prob, title, out_png):
    from sklearn.metrics import precision_recall_curve, average_precision_score
    try:
        prec, rec, _ = precision_recall_curve(y_true, y_prob)
        ap = average_precision_score(y_true, y_prob)
        plt.figure(); plt.plot(rec, prec, label=f"AP={ap:.3f}")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(title); plt.legend(loc="lower left"); plt.tight_layout()
        plt.savefig(out_png, dpi=150); plt.close()
    except Exception as e:
        J.log("plot", "warn", f"PR plot skipped: {e}")

# --- Train QSVM (precomputed kernel) ---
C = float(os.environ.get("QK_C", "5.0"))
clf = SVC(C=C, kernel="precomputed", probability=True, class_weight="balanced", random_state=0)
J.log("fit", "ok", f"QSVM: fitting on K_trtr (shape={K_trtr.shape}, C={C})")
clf.fit(K_trtr, ytr_sel)

# Choose threshold on validation (using K_vatr)
p_val = clf.predict_proba(K_vatr)[:,1]
thr = choose_threshold(y[va_idx], p_val, name="QSVM")

# Evaluate splits
splits = {
    "train": (K_trtr, ytr_sel),
    "val":   (K_vatr,  y[va_idx]),
    "test":  (K_tetr,  y[te_idx]),
}

rows, reports, cms = [], {}, {}
for split,(K,y_true) in splits.items():
    p = clf.predict_proba(K)[:,1]
    m, cm, rep = extended_metrics(y_true, p, thr)
    m.update({"model":"QSVM_kernel", "split":split})
    rows.append(m); cms[split]=cm; reports[split]=rep

# Save metrics

df_metrics = pd.DataFrame(rows)
df_metrics.to_csv(RESULTS/"metrics/qsvm_kernel_metrics.csv", index=False)

for split, cm in cms.items():
    save_cm_csv(cm, RESULTS/f"metrics/qsvm_kernel_cm_{split}.csv", normalized=False)
    save_cm_csv(cm, RESULTS/f"metrics/qsvm_kernel_cm_{split}_norm.csv", normalized=True)

with open(RESULTS/"metrics/qsvm_kernel_classification_reports.json","w",encoding="utf-8") as f:
    json.dump(reports, f, indent=2)

# Plots (test)
plot_roc(y[te_idx], clf.predict_proba(K_tetr)[:,1], "QSVM — ROC (test)", RESULTS/"plots/qsvm_kernel_roc_test.png")
plot_pr (y[te_idx], clf.predict_proba(K_tetr)[:,1], "QSVM — PR (test)",  RESULTS/"plots/qsvm_kernel_pr_test.png")

# Console + journal summary
row_test = df_metrics.loc[df_metrics["split"]=="test"].iloc[0]
print(row_test.to_string())

J.log(
    "eval", "ok",
    f"QSVM: test F1={row_test['f1']:.3f}, AUC={row_test['roc_auc']:.3f}, PR-AUC={row_test['pr_auc']:.3f}, thr={thr:.2f}"
)

# (Optional) Cell 6 — Per-dataset test diagnostics (if ds_idx available)

In [ ]:
# Evaluate generalization per dataset on TEST split only (reuse global thr)
if ds_idx is None:
    J.log("per-dataset", "warn", "ds_idx not found — skipping per-dataset diagnostics.")
else:
    rows = []
    uniq = sorted(np.unique(ds_idx[te_idx]))
    for d in uniq:
        name = ds_map.get(d, f"ds_{d}") if ds_map else f"ds_{d}"
        mask = (ds_idx[te_idx] == d)
        if mask.sum() == 0: 
            continue
        y_true = y[te_idx][mask]
        K_sub  = K_tetr[mask, :]  # rows subset; columns stay as train
        y_prob = clf.predict_proba(K_sub)[:,1]
        m, cm, _ = extended_metrics(y_true, y_prob, thr)
        m.update({"model":"QSVM_kernel", "dataset":name, "n": int(mask.sum())})
        rows.append(m)
    df_per = pd.DataFrame(rows)
    df_per.to_csv(RESULTS/"metrics/qsvm_kernel_per_dataset_test.csv", index=False)
    J.log("per-dataset", "ok", f"Saved per-dataset test metrics for {len(df_per)} dataset(s).")

# (Final) Cell 7 — Save run journal (what worked, what didn’t, and why)

In [ ]:
ts = time.strftime("%Y%m%d_%H%M%S")
base = RESULTS/"logs"/f"qsvm_kernel_{ts}"

issues = []
for e in J.events:
    if e["status"] in ("warn","fail"):
        issues.append(f"- [{e['step']}] {e['message']}")
rollup = "No warnings or failures." if not issues else "Issues observed:\n" + "\n".join(issues)
print("\n=== RUN SUMMARY ===\n" + rollup)

J.save(base)
(RESULTS/"logs"/f"qsvm_kernel_{ts}_summary.txt").write_text(rollup, encoding="utf-8")
print(f"📦 Metrics in: {RESULTS/'metrics'}  |  Plots in: {RESULTS/'plots'}  |  Kernels in: {RESULTS/'kernels'}")